In [1]:
%load_ext autoreload
%autoreload 2

import sys
import torch
from lifting_rl.linkage_env import LinkageEnv
from livelossplot import PlotLosses
import gym
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from lifting_rl.ddpg.utils import OUNoise, NormalizedEnv, Memory
from lifting_rl.reinforce.agent import ReinforceAgent
from lifting_rl.ddpg.utils import OUNoise, NormalizedEnv, Memory
from lifting_rl.ddpg.agent import DDPGagent
from lifting_rl.utils import NormalizedEnvVReinforce, NormalizedEnvVDDPG
from tqdm.auto import tqdm

In [2]:
params = {
    "N_LINKS": 5,
    "GOAL_POS":  np.array([np.pi / 4, np.pi/4, np.pi/4, -np.pi/2, np.pi/2, 0, 0, 0, 0, 0], dtype=np.float32),
    "INIT_STATE": np.array([np.pi / 2, np.pi/2, np.pi/2, -np.pi/4, np.pi/4, 0, 0, 0, 0, 0], dtype=np.float32),
    "PARAM_VALS": np.array([9.81, 0.4, 1, 0.4, 1, 0.6, 1, 0.4, 1, 0.4, 1]),
    "OBS_LOW": np.array([0, np.pi/4, -np.pi/2, -np.pi, -np.pi, -8, -8, -8, -8, -8], dtype=np.float32),
    "OBS_HIGH": np.array(
        [3*np.pi/4, 3*np.pi/2, 3*np.pi/4, np.pi/2, np.pi, 8, 8, 8, 8, 8], dtype=np.float32
    ),
    "ACT_LOW": -100,
    "ACT_HIGH": 100,
    "TIME_STEP": 0.01,
    "VIDEO_FPS": 30,
}

In [3]:
angles_file = "../data/skeleton_angles.csv"

In [4]:
# env = NormalizedEnv(LinkageEnv(angles_file, params, verbose=0))

In [5]:
env = gym.make("Pendulum-v0")
env.seed(42)
torch.manual_seed(42)

In [6]:
agent = DDPGagent(
    env, device = 'cuda:0', 
    hidden_size=128,
    actor_learning_rate=0.001,
    critic_learning_rate=0.001,
    gamma=0.99,
    tau=0.01,
    max_memory_size=2*50000,
    batch_size=64,
)
rewards = []
avg_rewards = []

liveloss = PlotLosses()
running_reward = 10
for episode in tqdm(range(10000)):
    state = env.reset()
    agent.reset()
    episode_reward = 0
    
    mean_mu = 0
    mean_std = 0
    
    for step in range(500):
        action, meta = agent.get_action(state, step)
        new_state, reward, done, _ = env.step(action)
        agent.finish_step(state, action, new_state, reward, done, meta)
        if episode % 50 == 0:
            env.render()
        episode_reward += reward
        #og_prob, entropy, mu, sigma_sq = meta
# #         mean_mu += mu
#         mean_std += sigma_sq
        
        if done:
            break
    
#     mean_mu = mean_mu / step
#     mean_std = mean_std / step
    
    running_reward = 0.05 * episode_reward + (1 - 0.05) * running_reward
    agent.update()
    
    metrics = {
        "running_reward": running_reward,
        "reward": episode_reward,
#         'mean_std': mean_std,
#         'mean_mu': mean_mu
    }
    liveloss.update(metrics)
    liveloss.send()
    
    rewards.append(episode_reward)
    avg_rewards.append(running_reward)

plt.plot(rewards)
plt.plot(avg_rewards)
plt.plot()
plt.xlabel('Episode')
plt.ylabel('Reward')
plt.show()

KeyboardInterrupt: 

In [ ]:


  running_reward = 10
    for i_episode in count(1):
        state, ep_reward = env.reset(), 0
        for t in range(1, 10000):  # Don't infinite loop while learning
            action = select_action(state)
            state, reward, done, _ = env.step(action)
            if args.render:
                env.render()
            policy.rewards.append(reward)
            ep_reward += reward
            if done:
                break

        running_reward = 0.05 * ep_reward + (1 - 0.05) * running_reward
        finish_episode()
        if i_episode % args.log_interval == 0:
            print('Episode {}\tLast reward: {:.2f}\tAverage reward: {:.2f}'.format(
                  i_episode, ep_reward, running_reward))
        if running_reward > env.spec.reward_threshold:
            print("Solved! Running reward is now {} and "
                  "the last episode runs to {} time steps!".format(running_reward, t))
            break
